In [1]:
system =  {"role": "system",
"content": f'''
你是一个事实核查专家，事实核查分为事实验证和解释生成两个任务。
你可以将事实验证理解为判断一个说法是否是真实的，将解释生成理解为对这个说法的真实性作出解释，表明这个说法为什么是对的/错的/证据不充分的。
现在我提供给你当前说法，一个黄金解释1，以及一个生成解释2。
你的任务是判断黄金解释1和生成解释2之间的一致性，并将其量化为一个分数。
这个分数有5个等级，即你打分的范围在1～5。
接下来我将给你具体分数等级的含义：
1:真实性不一致，例如黄金解释1里有“该说法是正确的”，而生成解释2里是“该说法是错误的”，并且解释内容也完全不相关。
2:真实性不一致，但解释内容相关。
3:真实性一致，解释内容不相关。
4:真实性一致，解释内容部分相关。
5:真实性一致，解释内容完全相关。
请你根据当前说法，一个黄金解释1，以及一个生成解释2量化一个一致性分数。
注意，你的输出只为一个阿拉伯数据，其范围为1～5。
注意：这里的完全相关不是指文本内容完全一样，是指意思一样，文本内容很相近即可。
'''} 

In [9]:
import requests 
import json
def gen_result(system,message,temperature):
    url = ""
   
    
    messages = [
            system,
            {"role": "user", "content": message},
    ]
    api_key = ""
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    
    model = 'gpt-4-0125-preview'#"gpt-4-0125-preview"#'GPT-3.5-Turbo-1106'#"gpt-4-0125-preview"#'GPT-3.5-Turbo-1106'#"gpt-4-0125-preview"#'gpt-4-0125-preview'#"GPT-3.5-Turbo-1106"#"gpt-4-1106-preview"
    
    data = {
        "model": model,
        "messages": messages,
        "temperature":temperature,
        # "tenant":"intelligentInteraction",
        # "velenOfflineChatFlag":"false"
    }
    content = json.dumps(data)
    
    try:
        response = requests.post(url, headers=headers, data=content)
        reply = response.json()['choices'][0]['message']['content']
        return 1,reply
    except Exception as e:
        print(e)
        return 0,""

def request_result(system,message,temperature):
    max_retry_nums = 3
    cnt = 0
    res = ""
    while(cnt<=max_retry_nums):
        flag,res = gen_result(system,message,temperature)
        if flag==1:
            try:
                # load_res = json.loads(res)
                break
            except:
                cnt+=1
        else:
            cnt+=1
            
    return res

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [187]:
import json

with open('../../data/CHEF-EG/test.json','r') as f:
    eval_data = json.load(f)
    # eval_data = [data['human_revision'] if data['human_revision'] !="" else data['explanation'] for data in eval_data]
    gold_explanation_list = [data['human_revision'] for data in eval_data]
    claim_list = [data['claim'] for data in eval_data]

now_path = '../test_result/stage2/Qwen2_stage2_train_ep4_rank128_lr3e-4/beam1/EGP_results.json'

with open(now_path,'r') as f:
    pred_dict = json.load(f)
    
pred_explanation_list = [pred_dict[str(i)] for i in range(len(eval_data))]

In [188]:
claim_list[0]

'空气中有碘化银漂浮物会危害健康。'

In [189]:
gold_explanation_list[0]

'证据1中明确指出，虽然碘化银被用作人工增雨(雪)的催化剂，但其使用量极小，每平方公里只有一克碘化银，并且是在很大的区域内分散使用，单位面积的含量微乎其微，几乎可以忽略不计。此外，济南市气象局官方微博回应称，在人工影响天气中使用的碘化银不会对环境和人体造成伤害。证据2中提到，产生的银离子含量远低于世卫组织标准和我国生活饮用水卫生标准，因此不会对人体健康构成威胁。综上所述，空气中的碘化银漂浮物在人工增雨(雪)作业中的使用量极小，且其对人体健康的影响远低于安全标准，因此该说法是错误的。'

In [190]:
pred_explanation_list[0]

'证据中明确指出，碘化银作为人工增雨（雪）的催化剂，在使用过程中，其释放的催化剂量非常少，且分布范围非常广，因此单位面积内的含量几乎可以忽略不计。此外，人工增雨（雪）是一个微物理过程，而不是产生新物质的化学变化。尽管碘化银中的银离子是重金属，但是在实际应用中使用的量极少，因此不会对人体和环境造成危害。因此，该说法是错误的。'

In [191]:
def get_user_message(claim,gold_explanation,pred_explanation):
    user_message = f'当前说法为：{claim}\n'
    user_message += f'黄金解释1为：{gold_explanation}\n'
    user_message += f'生成解释2为：{pred_explanation}'
    return user_message

In [192]:
scores = []
for idx,(claim,gold_explanation,pred_explanation) in enumerate(zip(claim_list,gold_explanation_list,pred_explanation_list)):
    if pred_explanation=='':
        continue
    res = request_result(system,get_user_message(claim,gold_explanation,pred_explanation),0.0)
    if res=='':
        continue
    scores.append(res)
    print(res)

5
5
5
5
4
5
5
5
5
2
5
5
5
4
2
4
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
4
5
5
5
2
5
5
5
5
2
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
2
5
2
5
5
5
5
5
5
4
5
4
2
5
5
3
5
5
5
5
5
5
4
5
4
2
5
5
5
5
5
2
5
5
5
5
2
5
5
5
5
5
5
5
5
1
5
5
2
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
4
5
5
4
5
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
2
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
5
5
5
4
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5


In [193]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9411255411255411

In [186]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.8477386934673368

In [179]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.842211055276382

In [172]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.8503778337531487

In [165]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9367965367965368

In [158]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.8498740554156172

In [18]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.8839195979899497

In [25]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.8909547738693467

In [32]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.7969773299748111

In [39]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.799497487437186

In [46]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9454545454545455

In [53]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.951304347826087

In [60]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9471861471861471

In [68]:
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.9139130434782609

In [110]:
###0
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.8517587939698492

In [89]:
###1
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.8547738693467336

In [96]:
###2
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.8537688442211055

In [103]:
###3
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.8537688442211055

In [75]:
###4
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.8597989949748744

In [82]:
scores = [int(score) for score in scores if len(score)==1 ]
sum(scores)/len(scores) * 0.2

0.8492462311557789

In [117]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9454545454545455

In [130]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9523809523809524

In [137]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.9541125541125542

In [144]:
scores = [int(score) for score in scores]
sum(scores)/len(scores) * 0.2

0.8316582914572864